<a href="https://colab.research.google.com/github/AlexCHEN-Engineer/CEE314HW/blob/main/Problemset5Q4ConductionTeaching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CEE 314 Computational Poromechanics, Spring 2022
Instructor : Prof. Ronaldo Borja 

TA: Wei Chen

# Configure the environment in google colab. 
Everyone with goole account can have access to the [google colab](https://colab.research.google.com/). Execute the first cell to get Fenics installed.

In [ ]:
%%capture
# Do not revise here. 
try:
    import google.colab
except ImportError:
    import ufl
    import dolfin
    import mshr
else:
    try:
        import ufl
        import dolfin
        import mshr
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import ufl
        import dolfin
        import mshr

# Import necessary finite element library
Import necessary finite element library, which includes `dolphin`, `mshr` and `numpy`. Here, `dolphin` is used to execute the finite element operation such as contructing the variational problem, assembling the equations and solve the linear system. `mshr` is a mesh library to deal with the mesh problems. `numpy` is similar to matlab and will only be used to plot the result.

In [ ]:
from dolfin import *
from mshr import *
import numpy as np
import matplotlib.pyplot as plt

## Generate mesh
We use the `Sphere` method to generate the mesh we need for this problem.

In [ ]:
# Create mesh and define expression
# Parameters
R = 1.

# Create geometry
s = Sphere(Point(0, 0, 0), 1)

# s2 = Sphere(Point(x, y, z), r)
b1 = Box(Point(0., 0., 0.), Point(1., 1., 1.))
geometry = s*b1

# Create mesh
mesh = generate_mesh(geometry, 15)
plot(mesh)

## Finite element formulation
In our current problem, the strong form ($S$) is \\
Given  $\bar{q}:\Gamma_q→\mathbb{R}$, find $h:\Omega→\mathbb{R}$ such that
\begin{equation}
\begin{aligned}
& \cal{S}\dot{h} = k\nabla^2 h. \\
& h = 0\quad \text{on} \quad \Gamma_h \\
& k∇h\cdot \boldsymbol{n} = \bar{q}\quad \text{on} \quad \Gamma_q \\
& h = h_0\quad \text{on}\quad \Omega\quad \text{when}\quad t=0 \\
\end{aligned}
\end{equation}

Recall the trial and weight functions in lectures, where we define:
\begin{equation}
\begin{aligned}
& \cal{T} = \{h|h\in H^1,\ h=0\quad \text{on}\quad \Gamma_h \}, \\
& \cal{W} = \{h|h\in H^1,\ w=0\quad \text{on}\quad \Gamma_h \}.\\
\end{aligned}
\end{equation}
Note that $\Gamma_h$ is the boundary with essential boundary conditions. $\Gamma_q$ is the boundary with natural boundary conditions. $\Gamma_h\cup \Gamma_q=\Gamma$.

Thus, we derive the weak form from the strong form.
\begin{equation}
\begin{aligned}
0 &= \int_{\Omega}\delta h(\cal{S}\frac{\partial h}{\partial t}-k\nabla^2h)dV\\
&= \int_{\Omega}\delta h \cal{S}\dot{h}dV -\int_{\Omega}\delta h k\nabla^2hdV \\
&= \cal{S}\int_{\Omega}\delta h \dot{h}dV +k\int_{\Omega}\nabla\delta h\cdot\nabla hdV -k\int_{\Gamma_q}\delta h\bar{q}dA\\
\end{aligned}
\end{equation}

To run the finite element code, you need to write out the weak form and complete it in the following cell.

Use backward time discretizations to yield numerical result. 
\begin{equation}
\cal{S}\int_{\Omega}\delta h \frac{h_{n+1}-h_n}{\Delta t}dV +k\int_{\Omega}\nabla\delta h\cdot\nabla h_{n+1}dV -k\int_{\Gamma_q}\delta h\bar{q}_{n+1}dA= 0
\end{equation}


# Implementation

In [ ]:
##############################
## TODO: Set the parameters ##
##############################
# You need to set different parameters to run the simulation!
# T = 1
# num_steps = 100

dt = T / (1.0*num_steps) #time step size

rho_f = 1000

E = 10.e7 #Pa
nu = 0.0

k = 1.15740741e-10 	#intrinsic permeability
mu_l = 1e-2 	#dynamic viscosity

lambda_= E*nu/(1+nu)/(1-2*nu)
mu = E/2/(1+nu)
K_e = E/3/(1-2*nu)

S = rho_f*9.8/K_e

In [ ]:
outer =  CompiledSubDomain("x[0]*x[0] + x[1]*x[1] + x[2]*x[2] > 0.9")
cutX =  CompiledSubDomain("near(x[0], side)", side = 0.0)
cutY =  CompiledSubDomain("near(x[1], side)", side = 0.0)
cutZ = CompiledSubDomain("near(x[2], side)", side = 0.0)

boundary = MeshFunction('size_t', mesh, mesh.topology().dim()-1)
boundary.set_all(1)
outer.mark(boundary, 2)
ds = Measure('ds', domain = mesh, subdomain_data = boundary)


V = FunctionSpace(mesh, 'P', 1)
bcp = DirichletBC(V, 0.0, outer) 
bcs=[bcp]

h_initial = Constant(100.)

# Define initial value.
h_n = interpolate(h_initial, V)
h_out = []
h_out.append(h_n(0, 0, 0))

# Define variational problem
# u and v represent the trial function space
# and the weight function space.
h = TrialFunction(V)
v = TestFunction(V)

schemes = {"Forward", "Backward", "Crank-Nicolson"}

def getVarition(scheme = "Backward"):
  if scheme not in schemes:
    raise Exception("Unknow schemes")

  if scheme == "Forward":
    return k*dt*dot(grad(h_n), grad(v))*dx + S*h*v*dx - S*h_n*v*dx

  if scheme == "Backward":
    return 0.5*k*dt*dot(grad(h+h_n), grad(v))*dx + S*h*v*dx - S*h_n*v*dx

  if scheme == "Crank-Nicolson":
    return k*dt*dot(grad(h), grad(v))*dx + S*h*v*dx - S*h_n*v*dx

F = getVarition("Backward")

a, L = lhs(F), rhs(F)

# create VTK file for saving solution 
# VTK file can be viewed in paraview. 
# vtkfile = File('CEE314HW5/solution.pvd')

In [ ]:
# define the variables to be solved.
h = Function(V)
from tqdm import tqdm_notebook
for n in tqdm_notebook(range(num_steps)):
    solve(a == L, h, bcs)
    # vtkfile << (u, t)
    # update the history value
    h_n.assign(h)
    h_out.append(h_n(0, 0, 0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/1000 [00:00<?, ?it/s]

### Postprocessing
Get the centerline pore pressure distribution after 11.57 days.

In [ ]:
time = np.linspace(0,100,1001)
plt.xlabel(r"time, $s$")
plt.ylabel(r"pore pressure, $kPa$")
plt.plot(time, h_out, 'k')